In [1]:
!pip install gradio --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 594.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/

In [8]:
import gradio as gr
import pandas as pd
import pickle
from datetime import datetime
from geopy.geocoders import Nominatim
from google.colab import drive
from geopy.point import Point
import requests
import json

drive.mount('/content/drive')

parking_info_df = pd.read_csv('/content/drive/My Drive/NUS-ISS AIS Projects/Project 2/Data/HDBCarparkInformation.csv')

# Load KNN model
#with open('knn_model_v2.pkl', 'rb') as f:
    #knn_model = pickle.load(f)

def get_singapore_locations():
    location_names = []
    base_url = "https://nominatim.openstreetmap.org/search"

    # Example query parameters
    params = {
        "q": "Singapore", #Search query for Singapore
        "format": "json",
        "countrycodes": "sg",  #Specify country code as Singapore
        "limit": 50,  # Increased limit to get more locations #Can be further increased if needed
        "bounded": 1 #To restrict results within the boundary of Singapore
    }

    response = requests.get(base_url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        try:
            data = response.json()
            for item in data:
                location_names.append(item["display_name"])
        except json.JSONDecodeError:
            print("Error decoding JSON response. Using fallback locations.")
            # Handle the error, e.g., use a default list of locations
            location_names = parking_info_df['address'].unique().tolist() #For fallback
    else:
        print(f"Request failed with status code: {response.status_code}. Using fallback locations.")
        location_names = parking_info_df['address'].unique().tolist() #For fallback

    return location_names

locations = get_singapore_locations()  # Store the list of locations

#locations = parking_info_df['address'].unique().tolist()  #Just in case, geolocator doesn't work we can use this to get input locations

parking_info_df = parking_info_df.rename(columns={'car_park_no': 'car_park_number'})
parking_info_df = parking_info_df.drop('gantry_height', axis=1, errors='ignore')

# Define a function to get the best parking location
def get_best_parking_location(input_location, input_time, car_park_type, free_parking, car_park_basement, night_parking):
    # Convert the input time to the required format (if needed)
    # Process location and time as model inputs
    #model_input = [input_location, input_time]  # Modify as necessary for model input format
    #prediction = knn_model.predict([model_input])

    #randomly select 10 car park ID
    prediction = parking_info_df['carpark_number'].sample(n=10).tolist()  #Randomly picked 3 10 locations to check working of UI

    print(prediction)

    #Get address of the predictions using parking_info_df
    address_list = parking_info_df[parking_info_df['carpark_number'].isin(prediction)]['address'].tolist()
    print(address_list)

    recommendations = []
    for parking_id in prediction:
        parking_details = parking_info_df[parking_info_df['carpark_number'] == parking_id].iloc[0].to_dict()  # Convert to dictionary
        details_string = f"Location: {parking_details['address']}, Type: {parking_details['car_park_type']}, Parking System: {parking_details['type_of_parking_system']},  Free Parking: {parking_details['free_parking']},  Night Parking: {parking_details['night_parking']},  Car Park Decks: {parking_details['car_park_decks']},  Basement Parking: {parking_details['car_park_basement']}"  # Format details
        recommendations.append(details_string)

        if (car_park_type == "Any" or parking_details['car_park_type'] == car_park_type) and \
           (free_parking == "Any" or parking_details['free_parking'] == free_parking) and \
           (car_park_basement == "Any" or parking_details['car_park_basement'] == car_park_basement) and \
           (night_parking == "Any" or parking_details['night_parking'] == night_parking):
            recommendations.append(parking_id)
            recommendations.append(details_string)

    # Get top 3 filtered recommendations
    top_3_recommendations = recommendations[:3]

    return ", ".join(recommendations)  # Convert list to comma-separated string

# Create Gradio interface
location_dropdown = gr.Dropdown(choices=locations, label="Select or Type Location", interactive=True)
time_input = gr.Textbox(label="Enter Time (HH:MM)", placeholder="e.g., 14:30")
car_park_type_dropdown = gr.Dropdown(choices=["Any", "Surface Car Park", "Multi-Storey Car Park"], label="Car Park Type")
free_parking_dropdown = gr.Dropdown(choices=["Any", "Yes", "No"], label="Free Parking")
car_park_basement_dropdown = gr.Dropdown(choices=["Any", "Yes", "No"], label="Basement Parking")
night_parking_dropdown = gr.Dropdown(choices=["Any", "Yes", "No"], label="Night Parking")
output_text = gr.Textbox(label="Best Parking Locations")

# Launch the Gradio interface

gr.Interface(
    fn=get_best_parking_location,
    inputs=[location_dropdown, time_input, car_park_type_dropdown, free_parking_dropdown, car_park_basement_dropdown, night_parking_dropdown],
    outputs=output_text,
    title="Best Parking Location Finder",
    description="Select a location and enter a time to find the best parking option available.",
).launch()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Request failed with status code: 403. Using fallback locations.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2dcb55d3bc8d1a47f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
